<a href="https://colab.research.google.com/github/katarinagresova/genomic_ML_playground/blob/main/Genomic_Language_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install biopython
!pip install fastai --upgrade

On March 15th I was not able to run SubwordTokenizer without installing this manualy first.

In [19]:
!pip install sentencepiece!=0.1.90,!=0.1.91

     |████████████████████████████████| 1.2MB 6.1MB/s 


In [2]:
!pip show fastai

Name: fastai
Version: 2.2.7
Summary: fastai simplifies training fast and accurate neural nets using modern best practices
Home-page: https://github.com/fastai/fastai/tree/master/
Author: Jeremy Howard, Sylvain Gugger, and contributors
Author-email: info@fast.ai
License: Apache Software License 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: pandas, scikit-learn, torchvision, torch, packaging, spacy, pip, fastcore, requests, pyyaml, scipy, fastprogress, matplotlib, pillow
Required-by: 


In [7]:
from Bio import SeqIO
from fastai.text.all import *

# Data preparation

Get data that we will use for our language model. For now, we will use human abinition cDNA, since it is small enough.

In [3]:
!wget http://ftp.ensembl.org/pub/release-103/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.abinitio.fa.gz
!gunzip Homo_sapiens.GRCh38.cdna.abinitio.fa.gz

--2021-03-15 06:23:18--  http://ftp.ensembl.org/pub/release-103/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.abinitio.fa.gz
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.139
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20601482 (20M) [application/octet-stream]
Saving to: ‘Homo_sapiens.GRCh38.cdna.abinitio.fa.gz’

Homo_sapiens.GRCh38 100%[===================>]  19.65M   654KB/s    in 32s     

2021-03-15 06:23:50 (633 KB/s) - ‘Homo_sapiens.GRCh38.cdna.abinitio.fa.gz’ saved [20601482/20601482]



Parse sequences from fasta file into list of sequences. Also put everything to lowercase in case there are mixed upper and lowercase. We don't want our model do learn that.

In [44]:
with open("Homo_sapiens.GRCh38.cdna.abinitio.fa", "rt") as handle:
  txts = L(str(record.seq).lower() for record in SeqIO.parse(handle, "fasta"))

We have 51756 sequences, together 64 739 432 characters.

In [45]:
print(len(txts))
print(len(''.join(txts)))

51756
64739432


Lets look at first sequence.

In [46]:
txts[0]

'atggaaagaggaaagaagaaaagaatttccaataagttacaacaaacttttcaccattctaaagaacccactttccttatcaaccaagctgggcttctctctagtgactcctattctagcctttccccagaaacagagagtgttaatcctggtgaaaatataaagacagacactcagaaaaagagacctgggactgtgatactatcaaaactgtcaagtagaagaattatatcggaaagccagcttagcccccctgtgatcccggcccgcaggcctggattccgggtatgctatatctgtggccgagaatttgggtcccagtcaattgccattcatgaaccccagtgcttgcagaagtggcatattgaaaacagcaagttgcccaagcatttgaggaggccagaaccctccaaaccacagtctctcagcagcagtgggtcctacagtcttcaggcaactaacgaggctgcatttcagagtgcccaggctcagctgctgccctgtgaatcctgtggccgcacattcttgccagatcatcttcttgttcatcacagaagctgcaagccaaagggtgagggtcccagagcaccacactcaaacagttctgatcatcttactggcctcaagaaagcttgtagtggaaccccagcccgaccaaggactgttatctgctacatatgtggtaaggaatttggcaccctgtcccttcctattcatgagcccaaatgcctggaaaagtggaaaatggaaaatgaccggctccctgtggagctccaccagccactcccacagaagcctcagccccttccgaatgcacagtccagccaagcgggaccaaatcaagctcagcttgtgttctgcccacattgtagccgaatctttacctcagaccgcctcctggtacaccagagaagttgtaaaactcatccttatgggccaaaatatcagaatttgaatttagggagtaaaggaggcctaaaagagtacactaattccaag

Take 1001st sequence for later testing.

In [47]:
txt = txts[1001]

For even quicker work, lets use just 1000 sequences for training.

In [48]:
txts = txts[:1000]

# Language model in fastai

## Tokenization

In [49]:
VOCAB_SIZE = 1000
tokenizer = SubwordTokenizer(vocab_sz=VOCAB_SIZE)
tokenizer.setup(txts)

{'sp_model': Path('tmp/spm.model')}

In [51]:
txt[:100]

'atgcatcagcagccttcggcaaaggggaaacaccgtgcagcagggctgacttggcaaagaggcacccccagaggaatgcgagggcttagagctgcaggca'

In [21]:
toks = first(tokenizer([txt]))
print(coll_repr(toks, 30))

(#140) ['▁atg','catca','gcag','ccttc','ggcaaa','gg','ggaaa','cacc','gtgcag','cagg','gctg','act','tggc','aaaga','ggcacc','cccag','aggaa','tgc','gaggg','cttag','agctg','caggca','atgcag','ttgct','acac','ggta','tctcag','tgt','caaga','atca'...]


Add Tokenizer on top of SubWordTokenizer. Not sure why this is needed, but I wasn't able to run it without this step.

I set `rules=[]` so no default rules will be applied - expecialy no encoding of repeating characters.

But maybe in future, some custom tokenizer with just special token for start of sequence would be nice. And for unkonown base - N.

In [57]:
tkn = Tokenizer(tokenizer, rules=[], sep='')
print(coll_repr(tkn(txt), 31))

(#140) ['▁atg','catca','gcag','ccttc','ggcaaa','gg','ggaaa','cacc','gtgcag','cagg','gctg','act','tggc','aaaga','ggcacc','cccag','aggaa','tgc','gaggg','cttag','agctg','caggca','atgcag','ttgct','acac','ggta','tctcag','tgt','caaga','atca','acag'...]


In [58]:
toks_all = txts.map(tkn)
toks_all[0]

['▁atgg',
 'aaaga',
 'ggaa',
 'agaaga',
 'aaagaa',
 'tttcca',
 'ataa',
 'gttac',
 'aaca',
 'aacttt',
 'tcacca',
 'ttct',
 'aaagaa',
 'ccca',
 'ctttcc',
 'ttat',
 'caa',
 'ccaag',
 'ctggg',
 'cttct',
 'ctct',
 'agtga',
 'ctccta',
 'ttcta',
 'gcct',
 'ttccc',
 'cagaaa',
 'cagag',
 'agtg',
 'ttaa',
 'tcctgg',
 'tgaaa',
 'atat',
 'aaaga',
 'cagaca',
 'ctc',
 'agaaa',
 'aagaga',
 'cctggg',
 'ac',
 'tgtga',
 'tactat',
 'caaa',
 'actg',
 'tcaagt',
 'agaaga',
 'att',
 'atat',
 'cgg',
 'aaagcca',
 'gctt',
 'agcc',
 'cccc',
 'tgtga',
 'tcccg',
 'gccc',
 'gcagg',
 'cctgg',
 'att',
 'ccggg',
 'tatg',
 'ctat',
 'atct',
 'gtggcc',
 'gaga',
 'attt',
 'gggt',
 'cccag',
 'tcaa',
 'ttgcc',
 'att',
 'catgaac',
 'cccag',
 'tgct',
 'tg',
 'cagaag',
 'tggcat',
 'attga',
 'aaa',
 'cagcaa',
 'gttg',
 'cccaag',
 'cattt',
 'gaggag',
 'gccag',
 'aacc',
 'ctcc',
 'aaac',
 'cacag',
 'tctct',
 'cagcagc',
 'agtgg',
 'gtcc',
 'tacag',
 'tctt',
 'cagg',
 'caac',
 'taa',
 'cgaggc',
 'tgc',
 'attt',
 'cagag',
 'tgccca',

## Numericalization

Let's translate tokens into numbers.

I have here representation of special 'xx' tokens. It is ok for now, but it would be nice to get rid of them in the future.

In [59]:
num = Numericalize()
num.setup(toks_all)
coll_repr(num.vocab,20)

"(#1000) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','tga','taa','tag','gca','cacc','ga','tcctca','ctga','tcag','atga','gtcc'...]"

This is numerical representation of our testing sequence

In [63]:
nums = num(toks)[:20]; nums

TensorText([162, 181, 170,  38, 689, 243,  53,  13, 878,  27, 208, 160, 149,  73,
        373,  21, 677, 147, 226, 843])

Let's put it back to letters.

In [64]:
''.join(num.vocab[o] for o in nums)

'▁atgcatcagcagccttcggcaaaggggaaacaccgtgcagcagggctgacttggcaaagaggcacccccagaggaatgcgagggcttag'

And compare with original sequence.

In [65]:
txt[:100]

'atgcatcagcagccttcggcaaaggggaaacaccgtgcagcagggctgacttggcaaagaggcacccccagaggaatgcgagggcttagagctgcaggca'

## DataLoader


In [66]:
nums_all = toks_all.map(num)

Creating LMDataLoader on top of numericalized tokenize_csv

This data loader should do propper splitting and padding for us. But parameters are just default for now. Investigation of more suitable parameter for genomics purposes is needed.

In [126]:
BATCH_SIZE = 64
dl = LMDataLoader(nums_all, bs=BATCH_SIZE, seq_len=72, shuffle=False)

In [127]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

This is our first intependent variable - what we show to the model.

In [75]:
' '.join(num.vocab[o] for o in x[0][:20])

'▁atgg aaaga ggaa agaaga aaagaa tttcca ataa gttac aaca aacttt tcacca ttct aaagaa ccca ctttcc ttat caa ccaag ctggg cttct'

And this is out first dependent variable - what we expect out model to predict. It is the same as independent variable, just shifted by one.

In [76]:
' '.join(num.vocab[o] for o in y[0][:20])

'aaaga ggaa agaaga aaagaa tttcca ataa gttac aaca aacttt tcacca ttct aaagaa ccca ctttcc ttat caa ccaag ctggg cttct ctct'

## Token + Numer + Loader

I got lost and wasn't able to make LMDataLoader work for training. But I was able to put all of previous step into few lines of code

In [133]:
splits = [list(range_of(txts[:800])), list(range(len(txts[:800]), len(txts)))]
tfms = [Tokenizer(SubwordTokenizer(vocab_sz=VOCAB_SIZE), rules=[], sep=''), Numericalize()]
dsets = Datasets(txts, [tfms], splits=splits, dl_type=LMDataLoader)

batch_size = 64
sequence_length = 72
dataloaders = dsets.dataloaders(bs=batch_size, seq_len=sequence_length, is_lm=True)

## Language model


In [107]:
class LMModel_LSTM(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers, p):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.drop = nn.Dropout(p)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h_o.weight = self.i_h.weight
        self.h = [torch.zeros(n_layers, BATCH_SIZE, n_hidden) for _ in range(2)]
        
    def forward(self, x):
        raw,h = self.rnn(self.i_h(x), self.h)
        out = self.drop(raw)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(out),raw,out
    
    def reset(self): 
        for h in self.h: h.zero_()

## Training

In [135]:
learn = TextLearner(dataloaders, LMModel_LSTM(VOCAB_SIZE, 64, 2, 0.4),
                    loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(15, 1e-2, wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,6.230824,5.303280,0.089740,00:31
1,4.892589,4.060067,0.106195,00:32
2,4.237363,3.882589,0.110922,00:31
3,3.946543,3.789527,0.112913,00:31
4,3.801299,3.751593,0.112347,00:31
5,3.724945,3.725109,0.112734,00:31
6,3.670852,3.707895,0.114819,00:31
7,3.644575,3.700676,0.114300,00:31
8,3.629875,3.695015,0.114149,00:31
9,3.609823,3.682459,0.115895,00:31
